# Custom Components

This is a guide on how to build a simple app and custom component spec
and launch it via two different schedulers.

See the [Quickstart Guide](quickstart.md) for installation and basic usage.

## Hello World

Lets start off with writing a simple "Hello World" python app. This is just a
normal python program and can contain anything you'd like.

<div class="admonition note">
<div class="admonition-title">Note</div>
This example uses Jupyter Notebook `%%writefile` to create local files for
example purposes. Under normal usage you would have these as standalone files.
</div>

In [1]:
%%writefile my_app.py

import sys
import argparse

def main(user: str) -> None:
    print(f"Hello, {user}!")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(
        description="Hello world app"
    )
    parser.add_argument(
        "--user",
        type=str,
        help="the person to greet",
        required=True,
    )
    args = parser.parse_args(sys.argv[1:])

    main(args.user)

Writing my_app.py


Now that we have an app we can write the component file for it. This
function allows us to reuse and share our app in a user friendly way.

We can use this component from the `torchx` cli or programmatically as part of a
pipeline.

In [2]:
%%writefile my_component.py

import torchx.specs as specs

def greet(user: str, image: str = "my_app:latest") -> specs.AppDef:
    return specs.AppDef(
        name="hello_world",
        roles=[
            specs.Role(
                name="greeter",
                image=image,
                entrypoint="python",
                args=[
                    "-m", "my_app",
                    "--user", user,
                ],
            )
        ],
    )

Writing my_component.py


We can execute our component via `torchx run`. The
`local_cwd` scheduler executes the component relative to the current directory.

In [3]:
%%sh
torchx run --scheduler local_cwd my_component.py:greet --user "your name"

torchx 2024-07-22 21:00:43 INFO     Tracker configurations: {}


torchx 2024-07-22 21:00:43 INFO     Log directory not set in scheduler cfg. Creating a temporary log dir that will be deleted on exit. To preserve log directory set the `log_dir` cfg option


torchx 2024-07-22 21:00:43 INFO     Log directory is: /tmp/torchx_8tuc8ai0


torchx 2024-07-22 21:00:43 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2024-07-22 21:00:44 INFO     Job finished: SUCCEEDED


local_cwd://torchx/hello_world-sw1wqcq96tsdg


If we want to run in other environments, we can build a Docker container so we
can run our component in Docker enabled environments such as Kubernetes or via
the local Docker scheduler.

<div class="admonition note">
<div class="admonition-title">Note</div>
This requires Docker installed and won't work in environments such as Google
Colab. If you have not done so already follow the install instructions on:
[https://docs.docker.com/get-docker/](https://docs.docker.com/get-docker/)</a>
</div>

In [4]:
%%writefile Dockerfile.custom

FROM ghcr.io/pytorch/torchx:0.1.0rc1

ADD my_app.py .

Writing Dockerfile.custom


Once we have the Dockerfile created we can create our docker image.

In [5]:
%%sh
docker build -t my_app:latest -f Dockerfile.custom .

#0 building with "default" instance using docker driver


#1 [internal] load build definition from Dockerfile.custom


#1 transferring dockerfile: 99B done


#1 DONE 0.0s


#2 [internal] load metadata for ghcr.io/pytorch/torchx:0.1.0rc1


...

#7 exporting layers


#7 exporting layers 4.0s done


#7 writing image sha256:6894810b99959b068fed4046cd9c709a785a92d740fbbc85f8b5e485bffc9500 done


#7 naming to docker.io/library/my_app:latest done


#7 DONE 4.0s


We can then launch it on the local scheduler.

In [6]:
%%sh
torchx run --scheduler local_docker my_component.py:greet --image "my_app:latest" --user "your name"

torchx 2024-07-22 21:02:34 INFO     Tracker configurations: {}


torchx 2024-07-22 21:02:34 INFO     Checking for changes in workspace `file:///home/runner/work/torchx/torchx/docs/source`...


torchx 2024-07-22 21:02:34 INFO     To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.


torchx 2024-07-22 21:02:35 INFO     Workspace `file:///home/runner/work/torchx/torchx/docs/source` resolved to filesystem path `/home/runner/work/torchx/torchx/docs/source`


torchx 2024-07-22 21:02:36 WARNING  failed to pull image my_app:latest, falling back to local: 404 Client Error for http+docker://localhost/v1.45/images/create?tag=latest&fromImage=my_app: Not Found ("pull access denied for my_app, repository does not exist or may require 'docker login': denied: requested access to the resource is denied")


...

torchx 2024-07-22 21:02:44 INFO     Built new image `sha256:523f4054b4be56bc1c1baea10daa9d9516480b017a3160a430061489a81f47a5` based on original image `my_app:latest` and changes in workspace `file:///home/runner/work/torchx/torchx/docs/source` for role[0]=greeter.


torchx 2024-07-22 21:02:45 INFO     Waiting for the app to finish...


greeter/0 Hello, your name!


torchx 2024-07-22 21:02:46 INFO     Job finished: SUCCEEDED


local_docker://torchx/hello_world-fzgbvxbtf7xqsd


If you have a Kubernetes cluster you can use the [Kubernetes scheduler](schedulers/kubernetes.rst) to launch
this on the cluster instead.


```sh
$ docker push my_app:latest
$ torchx run --scheduler kubernetes my_component.py:greet --image "my_app:latest" --user "your name"
```

## Builtins

TorchX also provides a number of builtin components with premade images. You can discover
them via:

In [7]:
%%sh
torchx builtins

Found 11 builtin components:


  1. dist.ddp


  2. dist.spmd


  3. metrics.tensorboard


  4. serve.torchserve


...

  7. utils.copy


  8. utils.echo


  9. utils.python


 10. utils.sh


 11. utils.touch


You can use these either from the CLI, from a pipeline or programmatically like
you would any other component.

In [8]:
%%sh
torchx run utils.echo --msg "Hello :)"

torchx 2024-07-22 21:02:48 INFO     Tracker configurations: {}


torchx 2024-07-22 21:02:48 INFO     Checking for changes in workspace `file:///home/runner/work/torchx/torchx/docs/source`...


torchx 2024-07-22 21:02:48 INFO     To disable workspaces pass: --workspace="" from CLI or workspace=None programmatically.


torchx 2024-07-22 21:02:48 INFO     Workspace `file:///home/runner/work/torchx/torchx/docs/source` resolved to filesystem path `/home/runner/work/torchx/torchx/docs/source`


torchx 2024-07-22 21:05:34 INFO     Building workspace docker image (this may take a while)...


...

torchx 2024-07-22 21:05:48 INFO     Built new image `sha256:05956815667f37fa6a1a6c84b0766e5df954714086c917274691e8e9d04a2820` based on original image `ghcr.io/pytorch/torchx:0.8.0dev0` and changes in workspace `file:///home/runner/work/torchx/torchx/docs/source` for role[0]=echo.


torchx 2024-07-22 21:05:48 INFO     Waiting for the app to finish...


torchx 2024-07-22 21:05:49 INFO     Job finished: SUCCEEDED


echo/0 Hello :)


local_docker://torchx/echo-wffl2x7dd7vsdc
